In [71]:
import os

os.environ.update({'AZURE_OPENAI_VERSION': '2024-02-15-preview'})
os.environ.update({'AZURE_OPENAI_DEPLOYMENT': 'hippo-gpt4'})
os.environ.update({'AZURE_OPENAI_ENDPOINT': 'https://hippooai.openai.azure.com/'})
os.environ.update({'AZURE_OPENAI_KEY': '53f889aad77847c2833fb1748900c18e'})
os.environ.update({'SERPER_API_KEY': '566584344afd2fb3277a5ab44690490efd1d5f99'})

In [62]:
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

from crewai import Agent
from crewai_tools import SerperDevTool
from crewai import Task
from crewai import Crew, Process

load_dotenv()

#azure_llm = AzureChatOpenAI(
#    azure_endpoint='https://hippooai.openai.azure.com/',
#    api_key='53f889aad77847c2833fb1748900c18e',
#    api_version='2022-12-01',
#    azure_deployment='hippo-gpt4'
#)

azure_llm = AzureChatOpenAI(
    azure_endpoint=os.environ.get('AZURE_OPENAI_ENDPOINT'),
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    api_version=os.environ.get('AZURE_OPENAI_VERSION'),
    azure_deployment=os.environ.get('AZURE_OPENAI_DEPLOYMENT'),
    openai_api_type='azure'
)

In [63]:
azure_llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x149700e10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x176781910>, openai_api_key='53f889aad77847c2833fb1748900c18e', openai_proxy='', azure_endpoint='https://hippooai.openai.azure.com/', deployment_name='hippo-gpt4', openai_api_version='2024-02-15-preview', openai_api_type='azure')

In [64]:
search_tool = SerperDevTool()

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True,
  llm=azure_llm
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False,
  llm=azure_llm
)

In [65]:
# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
  llm=azure_llm
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md',  # Example of output customization
  llm=azure_llm
)

In [69]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  llm=azure_llm,
  embedder={
                "provider": "huggingface",
                "config": {
                    "model": "mixedbread-ai/mxbai-embed-large-v1", # https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
                }
            }
)

2024-04-26 12:09:17,136 - 8213035712 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [72]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
print(result)



> Entering new CrewAgentExecutor chain...
To identify the next big trend in AI in healthcare, I need to conduct a thorough search on the internet to gather up-to-date information. I should focus on recent advancements, their potential benefits, possible risks, and market opportunities associated with them. Thus, I will use the "Search the internet" tool with a specific query related to the latest advancements in AI in healthcare.

Action: 
Search the internet

Action Input: 
{"search_query": "latest advancements in AI in healthcare"} 


Search results: Title: Contributed: Nine revolutionary ways AI is advancing healthcare
Link: https://www.mobihealthnews.com/news/contributed-nine-revolutionary-ways-ai-advancing-healthcare
Snippet: Contributed: Nine revolutionary ways AI is advancing healthcare ; 1. Medical imaging ; 2. Surgery ; 3. Medical research and data analysis ; 4. Drug ...
---
Title: Artificial Intelligence: How is It Changing Medical Sciences and Its ...
Link: https://www.ncb

2024-04-26 12:21:38,319 - 8213035712 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--f5b6a9381b41db232952979154faadfa2b634159e49d6b89b3d787cb15dca2e4


To begin crafting an engaging narrative on AI in healthcare, I need to gather more specifics on the latest trends and advancements in this field. I will focus on finding recent applications of AI, their potential benefits, and the associated risks.

Action: 
Search the internet

Action Input: 
{"search_query": "latest advancements in AI in healthcare"} 


Search results: Title: Contributed: Nine revolutionary ways AI is advancing healthcare
Link: https://www.mobihealthnews.com/news/contributed-nine-revolutionary-ways-ai-advancing-healthcare
Snippet: Contributed: Nine revolutionary ways AI is advancing healthcare ; 1. Medical imaging ; 2. Surgery ; 3. Medical research and data analysis ; 4. Drug ...
---
Title: Artificial Intelligence: How is It Changing Medical Sciences and Its ...
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7640807/
Snippet: Artificially intelligent computer systems are used extensively in medical sciences. Common applications include diagnosing patients, end-to

2024-04-26 12:22:17,889 - 8213035712 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--adc92096cc2d65c09bc90da141e344cbfa0011bc5d85ef694745ebc399ae2daa


Thought: 
The search results provide an overview of the ways AI is advancing healthcare as covered in the first article: 1) Medical imaging, 2) Surgery, 3) Medical research and data analysis, 4) Drug development, and 5) Early detection of fatal blood diseases. However, they lack detailed information about the pros, cons, and market opportunities. I'll proceed to the next article "Artificial Intelligence: How is It Changing Medical Sciences and Its..." to gather more details.

Action: 
Search the internet 

Action Input: 
{"search_query": "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7640807/"} 


Search results: Title: Artificial Intelligence: How is It Changing Medical Sciences and Its ...
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7640807/
Snippet: This article aims to present various aspects of AI as it pertains to the medical sciences. The article will focus on past and present day applications in the ...
---
Title: Overview of artificial intelligence in medicine - PMC - NC